In [33]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression

n_actions = 3
n_features = 20
n_data_list = [125, 250, 500, 1000, 2000, 4000, 8000, 16000, 32000, 64000]

Wx0 = np.random.normal(size=(n_actions, n_features))
Bx0 = np.random.normal(size=(n_actions, 1))

Wx = np.random.normal(size=(n_actions, n_features))
Bx = np.random.normal(size=(n_actions, 1))

Wxa_x = np.random.normal(size=(n_features,))
Wxa_a = np.random.normal(size=(n_actions,))

class Estimator():
    def __init__(self, n_data):
        self.x = np.random.normal(size=(n_data, n_features))
        self.data_generator()

    def sigmoid(self, x):
        return 1./(1.+np.exp(x))

    def feature2action0(self, x):
        prob_a = self.sigmoid(Wx0@x.T+Bx0).T
        prob_a = prob_a/prob_a.sum(axis=1, keepdims=True)
        return prob_a

    def feature2action(self, x):
        prob_a = self.sigmoid(Wx@x.T+Bx).T
        prob_a = prob_a/prob_a.sum(axis=1, keepdims=True)
        return prob_a

    def sample_actions(self, prob_a):
        data_size, action_size = prob_a.shape
        actions = np.array([np.random.choice(action_size, p=prob_a[i]) for i in range(data_size)])
        result = np.zeros_like(prob_a)
        result[np.arange(data_size), actions] = 1
        return result, actions

    def action2reward(self, x, a):
        return (Wxa_x@x.T+Wxa_a@a.T).T

    def data_generator(self):
        prob_a0 = self.feature2action0(self.x)
        a0, actions0 = self.sample_actions(prob_a0)
        r0 = self.action2reward(self.x, a0)

        prob_a = self.feature2action(self.x)
        a, actions = self.sample_actions(prob_a)
        r = self.action2reward(self.x, a)

        model = LogisticRegression(multi_class='multinomial', max_iter=200, solver='lbfgs')
        model.fit(X=self.x, y=actions0)

        a_pred0 = model.predict_proba(self.x)
        a_pred0 = a_pred0 / a_pred0.sum(axis=1, keepdims=True)
    
        self.prob_a = prob_a
        self.actions0 = actions0
        self.actions = actions
        self.a_pred0 = a_pred0
        self.r0 = r0
        self.r = r
    
    def DM_estimator(self):
        r0 = self.r0
        r = self.r
        a_pred0 = self.a_pred0
        prob_a = self.prob_a

        model = LinearRegression()
        model.fit(X=np.concatenate([self.x, a_pred0], axis=1), y=r0)

        self.r_pred = model.predict(X=np.concatenate([self.x, prob_a], axis=1))

        return abs(r.mean()-self.r_pred.mean())

    def IPS_estimator(self):
        r0 = self.r0
        r = self.r
        a_pred0 = self.a_pred0
        prob_a = self.prob_a
        actions = self.actions

        self.r_pred = np.array([
            r0_elm * prob_a_elm[actions_elm] / a_pred0_elm[actions_elm]
        for r0_elm, prob_a_elm, actions_elm, a_pred0_elm in zip(r0, prob_a, actions, a_pred0)])

        return abs(r.mean()-self.r_pred.mean())

In [38]:
for n_data in n_data_list:
    estimator = Estimator(n_data)
    print("DM", n_data, estimator.DM_estimator())
    print("IPS", n_data, estimator.IPS_estimator())

DM 125 0.034752591660133475
IPS 125 18.277038573949735
DM 250 0.010435457184294894
IPS 250 4.405918710833022
DM 500 0.002459303973889465
IPS 500 1.3308977175499326
DM 1000 0.010019238003185466
IPS 1000 6.349408494289956
DM 2000 0.029113310976599936
IPS 2000 1.1160223276163939
DM 4000 0.011529107973174791
IPS 4000 1.955571185172076
DM 8000 0.014488843593941224
IPS 8000 1.9671222308955227
DM 16000 0.014534955228839158
IPS 16000 1.4955066122513516
DM 32000 0.013419786770873979
IPS 32000 2.0777012005458833
DM 64000 0.01585505156706346
IPS 64000 1.981146316998527
